In [18]:
pip install opencv-python

  Using cached opencv_python-4.4.0.44-cp38-cp38-win_amd64.whl (33.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pandas.errors import EmptyDataError
import pandas as pd 
import numpy as np 
import os.path 
import cv2
from sklearn.neighbors import KNeighborsClassifier 

In [2]:
#function to convert data to csv  
f_name = "/Users/Nidhi/Desktop/Jupyter Notebooks/datasets/face_data.csv"
def write_to_csv(name, data): 
    if os.path.isfile(f_name): 
        print("File exists")
        df = pd.read_csv(f_name, index_col = 0) 
        latest = pd.DataFrame(data, columns = map(str, range(10000))) 
        latest["name"] = name 
        df = pd.concat((df, latest), ignore_index = True, sort = False) 
    else:  
        df = pd.DataFrame(data, columns = map(str, range(10000))) 
        df["name"] = name 
    df.to_csv(f_name) 

In [5]:
name = input("Enter you name: ")

Enter you name: Savitha


In [6]:
capture = cv2.VideoCapture(0)

In [7]:
classifier = cv2.CascadeClassifier("/Users/Nidhi/Desktop/Jupyter Notebooks/datasets/haarcascade_frontalface_default.xml")
#Cascade Classifier is an algo that takes a list of positive and negetive images to train itself and thus detects images

In [8]:
face_list=[]

In [9]:
count=0
while True:
    count=0
    captureStatus,frame = capture.read() #cap read return the frame and a boolean val
                                  #if the img is captured correctly
    if(captureStatus==False):
        continue
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)    
    faces = classifier.detectMultiScale(gray_frame,1.5,5)
    #print(faces)
    faces = sorted(faces,key=lambda face:face[2]*face[3])
    faces = faces[-1:] #last face i.e largest
    if len(faces)==1:
        face = faces[0]
        x,y,z,h = face
        cv2.rectangle(frame,(x,y),(x+z,y+h),(0,255,0),1)
        detected_face = frame[y:y+h,x:x+z] 
        cv2.imshow("Detected face",detected_face)
    cv2.imshow("full frame", frame)
    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'): #END LOOP
        break
    elif key & 0xFF == ord('c'): #capture image and append
        if len(faces)==1: 
            gray_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY)
            gray_face = cv2.resize(gray_face, (100, 100)) 
            face_list.append(gray_face.reshape(-1))  
        else: 
            print("face not found") 
        if len(face_list) == 10: 
            break
print(face_list)
face_list = np.array(face_list)
write_to_csv(name,face_list)
capture.release() 
cv2.destroyAllWindows() 

[array([161, 161, 157, ..., 108, 108,  95], dtype=uint8)]
File exists


In [4]:

df = pd.read_csv("/Users/Nidhi/Desktop/Jupyter Notebooks/datasets/face_data.csv")

In [5]:
df.head()

,Unnamed: 0,1.570000000000000000e+02,1.560000000000000000e+02,1.530000000000000000e+02,1.500000000000000000e+02,1.480000000000000000e+02,1.440000000000000000e+02,1.450000000000000000e+02,1.140000000000000000e+02,1.180000000000000000e+02,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,name
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,185.0,183.0,183.0,180.0,179.0,179.0,178.0,176.0,172.0,dh
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.0,27.0,15.0,12.0,14.0,15.0,14.0,16.0,15.0,Nidhi
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,171.0,141.0,156.0,189.0,152.0,137.0,108.0,108.0,95.0,Savitha


In [6]:
df.dropna(axis=1,inplace=True)

In [7]:
x_train =df.values[:,1:-1] 

In [8]:
y_train = df.values[:,-1]

In [9]:
x_train
y_train

array(['dh', 'Nidhi', 'Savitha'], dtype=object)

In [10]:
knnModel = KNeighborsClassifier(n_neighbors = 3)

In [11]:
knnModel.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=3)

In [12]:
captureImage = cv2.VideoCapture(0)

In [13]:
classifier = cv2.CascadeClassifier("/Users/Nidhi/Desktop/Jupyter Notebooks/datasets/haarcascade_frontalface_default.xml")

In [15]:
while True:
    captureStatus,frame = captureImage.read()
    if(captureStatus==False):
        continue
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)    
    faces = classifier.detectMultiScale(gray_frame,1.5,5)
    x_test = []
    for face in faces:
        x,y,z,h = face
        detectedFace = gray_frame[y:y+h,x:x+z]
        detectedFace = cv2.resize(detectedFace,(100,100))
        x_test.append(detectedFace.reshape(-1))
    if len(faces)>0:
        pred_y = knnModel.predict(np.array(x_test))
        for name,face in enumerate(faces):
            x,y,z,h = face
            cv2.rectangle(frame,(x,y),(x+z,y+h),(0,255,0),1)
            cv2.putText(frame,pred_y[name],(x-50, y-50), 
                              cv2.FONT_HERSHEY_DUPLEX, 2, 
                                         (0, 255, 0), 3)
    cv2.imshow("Full frame",frame)
    if cv2.waitKey(1)&0xFF == ord('q'):
        break

captureImage.release() 
cv2.destroyAllWindows()         
            
    